<a href="https://colab.research.google.com/github/vlx300/kb_colab/blob/master/Python_SQLite3_Example_Data_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Data Engineering** 
#**Relational vs Non-Relational Databases**

![alt text](https://ulriklyngs.com/sites/default/files/styles/final-blog-image-style/public/desat_chalkb.jpg?itok=7gumzPT8)

In [0]:
import sqlite3

In [0]:
db = sqlite3.connect(':memory:')  # using in memory database
cur = db.cursor()

##**Create three tables**

1.   **Customer**:  This table contains primary key, as well as customer first and last names
2.   **Items**: this table will contain the primary key, item name and item price
3.   **Items Bought**:this table contains the order#, date and price, it will also connect to the primary keys in items and customer tables. 



In [0]:
cur.execute('''CREATE TABLE IF NOT EXISTS customer (
  id integer PRIMARY KEY,
  firstname varchar(255),
  lastname varchar(255) )''')
cur.execute('''CREATE TABLE IF NOT EXISTS Item (
  id integer PRIMARy KEY,
  title varchar(255),
  price decimal )''')
cur.execute('''CREATE TABLE IF NOT EXISTS BoughtItem (
  ordernumber integer PRIMARY KEY,
  customerid integer,
  itemid integer, 
  price decimal,
  CONSTRAINT customerid
      FOREIGN KEY (customerid) REFERENCES Customer(id),
  CONSTRAINT itemid 
      FOREIGN KEY (itemid) REFERENCES Item(id) )''')

You passed a query to cur.execute() to create your three tables.. Now lets populate them with data 

In [0]:
cur.execute('''INSERT INTO Customer(firstname, lastname)
                VALUES ('Bob', 'Adams'),
                ('Amy', 'Smith'),
                ('Rob', 'Bennet');''')
cur.execute('''INSERT INTO Item(title, price)
                VALUES ('USB', 10.2),
                ('Mouse', 12.23),
                ('Monitor', 199.99);''')
cur.execute('''INSERT INTO BoughtItem(customerid, itemid, price)
                VALUES (1, 1, 10.2),
                (1, 2, 12.23),
                (1, 3, 199.99),
                (2, 3, 180.00),
                (3, 2, 11.23);''')  # Discounted Price


Now we have a few records in each table, you can use this data to answer a few more questions 

#**SQL Aggregation Functions**

Aggregation functions are those that perform mathematical operations  on a result set.  **AVG, COUNT, MIN, MAX and SUM**  Often you will need a **GROUP BY** or **HAVING** Clause to complement these aggregations. Let use **AVG** as a example: (See Below)

*AVG can compute the "mean" of a given resul*t

In [0]:
cur.execute('''SELECT itemid, AVG(price) FROM Boughtitem GROUP BY itemid''')
print(cur.fetchall())

[(1, 10.2), (2, 11.73), (3, 189.995)]


Here you have retrieved the averge price for each of the items bought in your database. you can see that the item with the item id# of 1 has an average price of $10.20

Lets make this easier to understand, by displaying the item name instead of item id#

In [0]:
cur.execute('''SELECT item.title, AVG(boughtitem.price) FROM Boughtitem as boughtitem
            INNER JOIN Item as Item on (item.id = boughtitem.customerid)
            GROUP BY boughtitem.itemid''')

print(cur.fetchall())

[('USB', 10.2), ('Monitor', 11.73), ('Mouse', 189.995)]


Another useful aggregation function is **SUM**. you can use this functon to display the total amount of money each customer spent (*See below*)

In [0]:
cur.execute('''SELECT customer.firstname, SUM(boughtitem.price) FROM BoughtItem as BoughtItem
            INNER JOIN Customer as Customer on (Customer.id = boughtitem.customerid)
            GROUP BY customer.firstname''')

print(cur.fetchall())

[('Amy', 180), ('Bob', 222.42000000000002), ('Rob', 11.23)]


#**Speeding up SQL Queries**

Speed depends on various factors but is mostly affected by how many of each of the following are present: 

*   **Joins**
*   **Aggregations**
*   **Traversals**
*   **Records**

the greater number of joins, the higher the complexity and the larger number of traversals in tables. Multiple joins are quite expensive to performs on several thousand records invloving several tables because *the database needs to cache the intermediate result*! At this point most people starting thinking about increasing DB memory size. Hmmm

![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRc8ZoZdP57kkw--eYUpsNhp-DnJOs7fblaUIgnFd4uMORRB9sATg&s)

Speed is also affected by whether or not there are indices present in the database or not. they are extrememly important and allow you to quick;ly searh thru a table to find a match for some column etc.  

Indices sort the records at th4 cost of higher insert time. as well as some storage Multiple columns can be combined to create a single index.  *Example. 'date' and 'price'columns can be combined because the query depends on both conditions.* 



#**Debugging SQL queries**

Most dstabases include an EXPLAIN QUERY PLAN that describes the steps the database takes to execute a query. for SQLite you can enable this functionality by adding EXPLAIN QUERY PLAN  in front of a SELECT statement 

In [0]:
cur.execute('''EXPLAIN QUERY PLAN SELECT customer.firstname, item.title,
                item.price, boughtitem.price FROM BoughtItem as boughtitem
                INNER JOIN Customer as customer on (customer.id = boughtitem.customerid)
                INNER JOIN  Item as item on (item.id = boughtitem.itemid)''')

print(cur.fetchall())

[(0, 0, 0, 'SCAN TABLE BoughtItem AS boughtitem'), (0, 1, 1, 'SEARCH TABLE Customer AS customer USING INTEGER PRIMARY KEY (rowid=?)'), (0, 2, 2, 'SEARCH TABLE Item AS item USING INTEGER PRIMARY KEY (rowid=?)')]


the query tries to list the first name, item title, original price for all bought items. 

**SQL Query Plan**

SCAN TABLE BoughtItem AS boughtitem
SEARCH TABLE Customer AS customer USING INTEGER PRIMARY KEY (rowid=?)
SEARCH TABLE Item AS item USING INTEGER PRIMARY KEY (rowid=?)

**NOTE:** the fetch statement in the python code only returns the explanation, **NOT THE RESULTS!**

#**Non-Relational databases** (Exp...NoSQL)

In the above section,  we laid out the differences between relational vs non-relational databases an used SQLite with python .  Now lets focus on NoSQL. We will use Mongo DB as example and use the same data as above to see the differences (*Laptop wont allow my connections thru FW, so we will just discuss*)

**Mongo DB is a document based No-SQL database and is Very Scalable**

In [0]:
import pymongo  # mongo Db comes pre-installed in Google COLAB

client = pymongo.MongoClient("mongodb://localhost:27017/")
# NOTE: this database wont be created until it is populated with data

db = client["example_database"]

customers = db["customers"]
items = db["items"]

customers_data = [{ "firstname": "Bob", "lastname": "Adams"},
                  { "firstname": "Amy", "lastname": "Smith"},
                  { "firstname": "Rob", "lastname": "Bennet"},]
items_data = [{ "title": "USB", "price": 10.2},
              { "title": "Mouse", "price": 12.23},
              { "title": "Monitor", "price": 199.99},]

customers.insert_many(customers_data)
items.insert_many(items_data)

#**NoSQL vs SQL**

Long term storage solutions

If you have a constantly changing schema,  such as finanncial regulatory information (GLBA SOX etc.)then No SQL can modify the records, and nest related information . Juat imagine the number of joins you would need to do in SQL if you had Eight orders of nesting.  Now if your goal is to "run reports" and extract information form the financial data, and ife conclusions? - in this case you would need to run complex queries which SQL tends to be faster doing.  so depends on what you need to get done. 

NOTE: SQL databases (PostgreSQL) has relases a feature that allows "querable JSON data" to be inserted as part of a record. Speed may be a concerns with this 

**Note: it is faster to query unstructured data from no-SQL Db than it is to query JSON fields from a JSON type column in PostgreSQL**. 

Speed isnt the only metric that matters. You also need to look at things such as **Transcations, Atomicity, durability anmd scalability**.  Transactions are important for financial applications.



*   **Elastic Search**: highly efficient in Text searches. Document based database to crate a powerful searh tool.
*   **Newt DB**: combines ZODB and PostgreSQL JSON feature to create a Python friendly No-SQL DB
*   **Influx DB** : time seried Db to store events. 





#**Cache databases**

a cache database is a very fast, short term storage solution for shorth lived structed and unstructured data. It can be partinioned and scaled according to you specific needs and requirements. typically smaller in size than your main DB and becuase of this your cache databse normally resides in memory bypassoing the need to read from disk. 

the normally live alongside your main SQL and No-SQL databases with the goal of alleviate load and speed response times
###**REDIS  Example**

If you ve ever used python dictionaries, then REDIS follows the same structure. Is a key-value store where you can SET and GET just like a python DICT

In [0]:
pip install redis

In [0]:
import redis
from datetime import timedelta

In [0]:
# in a real web application, configuration is obtained via settings or utils 
r = redis.Redis()

In [0]:
# Assume this is a getter handling a request 
def get_name(request, *args, **kwargs):
  id = request.get('id')
  if id in r:
    return r.get(id)  # Assume we have an {id: name} store
  else:
    # get some data from the main DB here, we assume we already did it
    name = 'Bob'
    # set the valule in the cache database, with expiration time
    r.setex(id, timedelta(minutes=60), value=name)
    return name

#**Design Patterns and ETL Concepts**

In larfge applications, you will oftern use mor ethan on type of database. I fact, its possibloe to utilize PostgreSQL, MongoDB and Redis all within one application. One challenging problem is state changes betweeen databases. this exposes the developer to issue of consistency . Conside rthe below example:

1. A value in database #1 is updated
2. that same value in database #2. Is NOT updated. 
3. A query is run on database #2

Now we have outdated results. The result returned from databse #2 wont reflect the updated data from database #1. This can happend with any two databases. **This is especially common is the main database is NoSQL**. Exp MongoDB. and the data is bring transformed into a SQL database (sqlite) for query purposes.

Databses have "backroun-workers" to tackle such issues.   These workers **EXTRACT** data from one database, **TRANSFORM** it in some way and then **LOAD** it into the target database.   When converting froim the NoSQL databse to SQL database, the ETL process takes th following steps:

1. **EXTRACT**:  there is a MONGO Db trigger whenever a record is created/updated and so on. A callback functioned is called "Asynchronously" on a seprate thread. 

2. **TRANSFORM**: Parts of the record are extracted, normalized, and put into the correct data structure "Row"to be inserted into SQL.

3. **LOAD**: the SQL database is updated in batches or as a single record for High-Volume writes

this workflow is quite common for financial, gaming and reporting applications. In these cases, the constantly changing schema requires a NoSQL database, but reporting, analysis and aggregation requires a SQL database. 






#**ETL Challenges**

There are several challenging concepts in ETL:

1. **Big-Data**
2. **Stateful problems**
3. **Asynchronous Worker**s
4. **Type-Matching**

and the list goes on. However since the steps in ETL proxess are well-defined and logical, the data and backend engineers will typically worry more about performance and availability rather than implementation. 

If you application is writing thousands of records per second to MongoDB, then your ETL worker neds to keep up with transforming, loading nd delivering the data to the user in  requested form. Speed and latency can become an issue.   Typically these worker will be written in fast languages. You can use compiled code for the Transform step.you may want to consider. 

**NOTE**:  Multi-processing and separation of workers are other solutions 
Consider Utilizing Numba (http://numba.pydata.org/) for CPU intensive functions.  



#**Design Patterns in Big-Data**

when you have terabytes of data, you will need mutiple machine sot handle all of that data. a database aggregation funciton can be a very complex operation. How can you query, aggregate and make use of relatively big data in a efficient way?

##**Apache Map-Reduce**
Introduced by apache foundation,  this follow the Map, Shuffle, Reduce workflow.  the idea is to map different data on separate machines (**Clusters**), then you can perform work on the data, grouped by a key, and finally aggregate thr data at the final stage. This workflow is still in use today, but has been fading recently in favor of **Apache Spark**  https://spark.apache.org/

##**Common Aspects of ETL process and Big Data workflows**

Both workflows ollow the Producer-Consumer pattern. A worker (The Producer) produces data of some  kind. and outputs to the pipeline. this pipeline can take many forms including networ messages and triggers. After the producer outputs the data , the consumer consumes and make use of the data.  these worker typically work in an asynchronous manne and executed via seperate proceses. 

the producer is like the **Extract** and **Transform** Steps in the above ETL process. Similarly in Big Data, the Mapper can be seen as the Producer, and the Reducer is effectively the consumer. This seperation of concerns is extremely important in the development and architecure design of applications  